<a href="https://colab.research.google.com/github/Nivz-28/File-Converter/blob/main/customer_support_colab_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enhanced Customer Support Agent Tutorial

Welcome to the interactive Google Colab tutorial for building an intelligent customer support agent using **LangGraph** and **HuggingFace** models. Follow along with narrative, code cells, and explanations.

## Prerequisites

Before you begin, ensure you have:
A CUDA-enabled GPU runtime in Colab for faster inference.

## 🚀 Setup

In [4]:
!pip install -q transformers bitsandbytes langgraph langchain-core torch langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


## 🔑 1. Configuration & Logging

In [5]:
import os
import logging

os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s'
)
logger = logging.getLogger('CustomerSupportAgent')

## 🤖 2. Load Quantized LLM

In [6]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
from langchain.llms import HuggingFacePipeline

gpu_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16
)
model_name = 'bitext/Mistral-7B-Customer-Support-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=gpu_config,
    device_map='auto',
    torch_dtype=torch.float16
)

pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device_map='auto',
    do_sample=False,
    max_new_tokens=256,
    return_full_text=False
)
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

<ipython-input-6-63812980>:33: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


## 🛠 3. Utilities & State Definition

In [7]:
from typing import Dict, TypedDict, Callable
import functools
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain import LLMChain

def cache(func: Callable) -> Callable:
    store = {}
    @functools.wraps(func)
    def wrapper(state: Dict) -> Dict:
        key = (func.__name__, state['query'])
        if key not in store:
            store[key] = func(state)
        return store[key]
    return wrapper

def build_chain(template: str) -> LLMChain:
    prompt = ChatPromptTemplate.from_template(template)
    return LLMChain(llm=llm, prompt=prompt)

class State(TypedDict):
    query: str
    category: str
    sentiment: str
    response: str

## 🔍 4. Query Classification & Sentiment Analysis

In [8]:
CATEGORIES = ['Technical','Billing','Account','Shipping','Returns','Product Info','General']
SENTIMENTS = ['Strongly Positive','Positive','Neutral','Negative','Strongly Negative']

@cache
def categorize(state: State) -> State:
    template = (
        f"Classify into one of: {', '.join(CATEGORIES)}. Respond with category only.\n\n"
        "Query: {query}"
    )
    raw = build_chain(template).run(query=state['query'])
    match = re.search(rf"\b({'|'.join(CATEGORIES)})\b", raw)
    return {'category': match.group(1) if match else 'General'}

@cache
def analyze_sentiment(state: State) -> State:
    template = (
        f"Choose one: {', '.join(SENTIMENTS)}. Respond with label only.\n\n"
        "Message: {query}"
    )
    raw = build_chain(template).run(query=state['query'])
    match = re.search(rf"\b({'|'.join(SENTIMENTS)})\b", raw)
    return {'sentiment': match.group(1) if match else 'Neutral'}

## 💬 5. Response Handlers & Escalation

In [15]:
HANDLER_INSTRUCTIONS = {
    'Technical': 'Technical support: ...',
    'Billing':   'Billing support: ...',
    'Account':   'Account support: ...',
    'Shipping':  'Shipping support: ...',
    'Returns':   'Returns support: ...',
    'Product Info': 'Product Info: ...',
    'General':   'General support: ...'
}

def make_handler(instr: str) -> Callable:
    @cache
    def handler(state: State) -> State:
        prompt = f"{instr}\n\nUser: {{query}}\n\nDo not include placeholders."
        resp = build_chain(prompt).run(query=state['query']).strip()
        return {'response': resp}
    return handler

for cat, instr in HANDLER_INSTRUCTIONS.items():
    globals()[f'handle_{cat.lower().replace(" ", "_")}'] = make_handler(instr)

@cache
def escalate(state: State) -> State:
    return {'response': 'This request has been escalated to a specialist.'}
    # Routing logic based on sentiment
def route_query(state: State) -> str:
    if state["sentiment"] == "Strongly Negative":
        return "escalate"
    return state["category"]

## 🔄 6. Workflow Assembly

In [16]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(State)
# Add core nodes
workflow.add_node('categorize', categorize)
workflow.add_node('analyze_sentiment', analyze_sentiment)

# Add all category-specific handler nodes
for cat in CATEGORIES:
    func_name = f'handle_{cat.lower().replace(" ", "_")}'
    workflow.add_node(cat, globals()[func_name])

# ✅ Add escalate node BEFORE routing
workflow.add_node('escalate', escalate)

# Routing logic
workflow.add_edge('categorize', 'analyze_sentiment')
workflow.add_conditional_edges(
    'analyze_sentiment',
    route_query,
    {cat: cat for cat in CATEGORIES} | {"escalate": "escalate"}
)

# Add edges to END
for node in CATEGORIES + ['escalate']:
    workflow.add_edge(node, END)

# Entry point
workflow.set_entry_point('categorize')
app = workflow.compile()


## ▶️ 7. Run & Test

In [20]:
def run_customer_support(query: str) -> Dict[str, str]:
    """
    Runs the customer support workflow on a given query and returns results.
    """
    out = app.invoke({'query': query})
    print(
        f"Query: {query}\n"
        f"Category: {out.get('category', '')} | Sentiment: {out.get('sentiment', '')}\n"
        f"Response: {out.get('response', '')}\n"
        + "-" * 60
    )

tests = [
    'Error 503 when pasting text in Ubuntu.',
    'Unexpected $49 charge on invoice.',
    'Password reset loop.',
    'Order #B98765 in transit?',
    'Received cracked screen.',
    'Supports HDR10+ and 144Hz?'
]

for q in tests:
    print(run_customer_support(q))

Query: Error 503 when pasting text in Ubuntu.
Category: Technical | Sentiment: Negative
Response: Assuredly! I'm here to assist you with the error you're encountering when pasting text in Ubuntu. Error 503 can occur due to various reasons, such as network connectivity issues, software bugs, or compatibility problems. To troubleshoot this issue, I recommend the following steps:

1. Check your internet connection: Ensure that you have a stable internet connection. Try disconnecting and reconnecting to your network or switching to a different network if possible.

2. Update your software: Make sure that your Ubuntu system and the applications you're using are up to date. You can check for updates by opening the terminal and typing `sudo apt update` followed by `sudo apt upgrade`.

3. Check for software conflicts: If you have recently installed or updated any software, try uninstalling or reverting the changes to see if the error resolves.

4. Try a different text editor: If the issue pers